In [ ]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
from sklearn.preprocessing import StandardScaler
import csv
from datetime import datetime
import tflite_runtime.interpreter as tflite

In [ ]:
# TFLite 모델 및 스케일러 로드
interpreter = tflite.Interpreter(model_path="./model/cnn1d_model_compatible.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

scaler = joblib.load('./model/scaler_cnn.joblib')

In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c) # (아날로그 → 디지털 변환기) 객체
ads.gain = 2/3 # 증폭 배율 설정

# ADS1115 채널 선택
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 센서 보정값
ZMPT_offset = 2.5539
ZMPT_scale = 997.6

ACS712_offset = 2.5087
ACS712_sensitivity = 0.0990

In [ ]:
# 파생 피처 추출 함수
def extract_features_from_sequence(seq):
    voltage_seq, current_seq = zip(*seq)
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [ ]:
# CNN 입력 시퀀스 생성
def build_cnn_input(data_buffer, seq_len=8):
    if len(data_buffer) < seq_len + 7:
        return None

    sequence = []
    # 마지막 8개의 연속 시퀀스를 뽑기 위한 시작점 조정
    for i in range(len(data_buffer) - (seq_len + 7), len(data_buffer) - 7):
        window = data_buffer[i:i+8]
        if len(window) < 8:
            continue  # 예외 방지
        voltage_seq, current_seq = zip(*window)
        features = extract_features_from_sequence(list(zip(voltage_seq, current_seq)))
        sequence.append(features)

    if not sequence:
        return None
    return np.array(sequence)


In [ ]:
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    
    # 정확히 연속된 1이 min_consecutive 이상인지 확인
    count = 0
    for val in reversed(anomaly_buffer): # 최근 값부터 역순으로 확인
        if val == 1:
            count += 1
        else:
            break
    
    return count >= min_consecutive

In [ ]:
# 테스트용 CSV 데이터 (센서 미사용시)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('../../data/test/test_dataset_187.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)
csv_filename = "./log/cnn1d_log5(0506).csv"

In [ ]:
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    USE_SENSOR = True
    BUFFER_SIZE = 15  # 데이터 버퍼 크기 상수로 정의
    ANOMALY_THRESHOLD = 1
    ANOMALY_THRESHOLD_VALUE = 0.48  # 임계값 상수로 정의

    data_buffer = []
    anomaly_buffer = []
    first_anomaly_detected = False
    anomaly_start_time = None
    prev_voltage = 0  # 초기값 설정
    prev_current = 0
    start_time = time.perf_counter()
    idx = -1

    # CSV 기록
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        while True:
            idx += 1

            if USE_SENSOR:
                try:
                    v_raw_sensor = voltage_ch.voltage
                    c_raw_sensor = current_ch.voltage
                    voltage = (v_raw_sensor - ZMPT_offset) * ZMPT_scale
                    current = (c_raw_sensor - ACS712_offset) / ACS712_sensitivity

                    prev_voltage = voltage
                    prev_current = current

                except (OSError, ValueError, RuntimeError) as e:
                    print(f"⚠️ 센서 오류 발생: {e}")
                    print("🩹 이전 정상값을 사용합니다.")
                    voltage = prev_voltage if 'prev_voltage' in locals() else 0
                    current = prev_current if 'prev_current' in locals() else 0

            else:
                voltage = df['voltage'].iloc[idx]
                current = df['current'].iloc[idx]

            data_buffer.append((voltage, current))

            anomaly = 0

            if len(data_buffer) >= BUFFER_SIZE:
                data_buffer.pop(0)

            # CNN 입력 생성 및 추론
            if len(data_buffer) >= BUFFER_SIZE:
                X_seq = build_cnn_input(data_buffer)
                if X_seq is None:
                    continue
                X_scaled = scaler.transform(X_seq)
                X_scaled = X_scaled.reshape(1, 8, 8)

                interpreter.set_tensor(input_details[0]['index'], X_scaled.astype(np.float32))
                interpreter.invoke()
                output_data = interpreter.get_tensor(output_details[0]['index'])
                y_pred = output_data[0][0]
                anomaly = int(y_pred > ANOMALY_THRESHOLD_VALUE) # 예측 퍼센트

                anomaly_buffer.append(anomaly)
                if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                    anomaly_buffer.pop(0)

                if anomaly == 1 and anomaly_start_time is None:
                    anomaly_start_time = time.perf_counter()
                elif anomaly == 0:
                    anomaly_start_time = None

                if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                    print("⚡ 아크 이상 감지!", idx)
                    if not first_anomaly_detected and anomaly_start_time is not None:
                        total_elapsed = time.perf_counter() - start_time
                        actual_elapsed = time.perf_counter() - anomaly_start_time
                        print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                        print(f"⏱️ 연속 {ANOMALY_THRESHOLD}개 이상 신호까지 소요 시간: {actual_elapsed:.6f}초")
                        first_anomaly_detected = True
                        timestamp = datetime.now().isoformat()
                        writer.writerow([timestamp, voltage, current, anomaly])
                        break

                timestamp = datetime.now().isoformat()
                writer.writerow([timestamp, voltage, current, anomaly])

            # 상태 출력
            cpu_usage = psutil.cpu_percent(interval=0.0)
            memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
            print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
            print(f"v_raw:{voltage:.4f}, current_raw:{current:.4f}, anomaly:{anomaly}")

            time.sleep(0.001163)  # 860Hz 기준

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

🚀 실시간 아크 감지 시작합니다!
🧠 메모리: 173.39MB | 🧮 CPU: 3.10% | idx: 0
v_raw:-16.9841, current_raw:0.3831, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 0.00% | idx: 1
v_raw:-8.1928, current_raw:0.3793, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 40.00% | idx: 2
v_raw:-48.9697, current_raw:0.3812, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 0.00% | idx: 3
v_raw:-8.3798, current_raw:0.3774, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 12.50% | idx: 4
v_raw:-11.3726, current_raw:0.3869, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 0.00% | idx: 5
v_raw:-4.4518, current_raw:0.3850, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 0.00% | idx: 6
v_raw:-13.0561, current_raw:0.3850, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 11.10% | idx: 7
v_raw:-10.8115, current_raw:0.3888, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 0.00% | idx: 8
v_raw:-9.5021, current_raw:0.3869, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 11.10% | idx: 9
v_raw:-17.9194, current_raw:0.3869, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 11.10% | idx: 10
v_raw:-16.0489, current_raw:0.3888, anomaly:0
🧠 메모리: 173.39MB | 🧮 CPU: 12.50% |